# 🔍 Objective:

This project will introduce you to Retrieval Augmented Generation and how it can be used to expand the knowledge base of an existing pretrained LLM.

# 🧠Glossary:

We're going to be using some rather fancy sounding words that you may not have come across before. I'd highly reccommend googling them, but feel free to refer to this cheat sheet if you forget.


1.   Vector: Think of this as a list of numbers specifically used to represent co-ordinates. So a vector containing the co-ordinates to a point at (2,8) would be [2, 8]. Now we'll often be using arrays of vectors, we'll also be using a library called numpy, which supports 2D arrays (basically an array of arrays) much better than regular Python.

2.   Embeddings:

*   A huge part of RAG is semantic search (i.e. searching by meaning). This is the core of what makes RAG so powerful, as it can tell us how similar two sentences are based on what they mean, even if they are worded differently.

*   The way this is done is by using an embedding model to convert text (or images) to points in space. The closer two points are, the closer in meaning their corresponding texts are.

*   We will be storing the co-ordinates of the points in vectors, hence the need for a vector database.

*   Note: While it is helpful to think of the points generated by an embedding model as points in 3 dimensions, most embedding models generate points in higher dimensions; the model we are using generates points in 384 dimensions! So instead of our points having an x, y and z co-ordinate, they will have x, y, z, w, v,..... co-odinates.

3. LLM: Large Language Models (LLM) can generate text based on a provided prompt. Sound familiar? It should; ChatGPT is a Large Language Model! We'll be using an LLM to actually answer a users question. The problem here is that LLMs aren't all knowing; they can only answer questions based on what they've been trained on. A way to remedy this is to use techniques like RAG to work out what snippets of text from an external source are most similar to the users question and feeding them to the LLM along with the users question. The LLM can use this data to generate a natural sounding answer.

4. Chunks:
* A chunk is simply a smaller piece of a larger piece of text. The reason you'd want to break down, say, a book or a pdf is to find relevant pieces of information in it. After all, we don't want to throw an entire PDF at our LLM and have it decipher all of it, now do we?

* We'll use a bunch of functions that we'll get to later to work out which chunks are the most relevant to our question and pass just those to our LLM.

* The size of a chunk is entirely up to the programmer, but you want a chunk to be big enough for it to actually have some sort of meaning in it on its own, while not being big enought to contain too many different kinds of information in it.


# 📌 What You'll Do:



1.   Create a rudimentary vector database by defining the functions in the template class below
2.   Convert a pdf of a stock trading tutorial to a bunch of small strings (called chunks. we'll get to why later).
3.   Use this vector database to store the embeddings of the chunks.
4. Handle user queries by querying our database for the most relevant chunks and feeding them to our LLM to generate an output.

In [1]:
#library installations. we'll be using sentence_transformers for our embedding model and pypdf to read a pdf of our choosing.

!pip install pypdf
!pip install sentence_transformers

#fun fact: the ! is used to signify that these are shell commands

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import numpy
import transformers
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader as PDFReader
from transformers import pipeline
from huggingface_hub import login

#I highly recommend sticking to this LLM; its small and runs decent in a colab notebook, especially if you have GPU acceleration enabled.
LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#Same goes for our embedding model.
EMBEDDING_MODEL = "all-MiniLM-L12-v2"
HF_API_KEY = "YOUR-HF-API-KEY-HERE"#Not a necessity.

In [ ]:
login(token=HF_API_KEY)

In [4]:
class VectorDB:
  embedModel = None#we will be using embedModel to refer to our embedding model through out this class. Its value is initialized in the constructor

  nDim = 0#number of dimensions. Remember how I told you the embedding model generates points in space?
          #nDim will contain the number of dimensions of the space these points are generated in. It will come in handy later

  _embeddings = None#This is the numpy array that will actually store our embeddings. the underscore at the start of the variable name
                    #signifies that this a private variable; i.e it should not be directly accessed outside of this class.

  _strings : list[str] = []#we will also be storing the strings that we are generating embeddings for.


  #note: _embeddings is a numpy array. THIS IS NOT THE SAME AS A REGULAR PYTHON ARRAY.
  #      _strings on the other hand is a regular ole Python array.



  def __init__(self, model_name : str) -> None:#DO NOT CHANGE. This is the constructor funtion.
    #we've set up a lot of the boilerplate code for you; this code instantiates all the class variables and loads the embedding model.
    self.embedModel = SentenceTransformer(model_name)
    self.nDim = self.embedModel.get_sentence_embedding_dimension()

    self._embeddings = numpy.ndarray((0, self.nDim), dtype = numpy.float32) #sets _embeddings to an array of an array.
    #The first number (0) represents the number of arrays that are stored in _embeddings. 0 for now as its empty.
    #nDim refers to the number of values in each of those subarrays. We need one to represent each dimension, the same way you'd need an array with 3 values to represent a 3d point.

    self._strings = []

  def addToDatabase(self, input : list[str]):
    '''
    Your code should do two things here:
    1. Convert the strings in the input array to embeddings and add them to the _embeddings array
    2. Store their corresponding strings (IN THE SAME ORDER) in the _strings array

    Hint: the documentation for the SentenceTransformers library can be found here: https://sbert.net/docs/package_reference/sentence_transformer/index.html
    If you want to know which function to call for anything related to the embedding model (eg: generating embeddings) have a look at this.
    '''
    #Your code goes here!
    embeddings = self.embedModel.encode(input)
    self._embeddings = numpy.vstack([self._embeddings, embeddings])
    self._strings.extend(input)

    #The pass command will cause everything within this function thats after pass to be ignored.
    #Delete it once you start coding or keep all your code above it


  def clearDatabase(self):
    '''
    This function should clear the database by emptying the _embeddings and _string arrays.
    '''
    #Your code goes here!
    self._embeddings = numpy.ndarray((0, self.nDim), dtype=numpy.float32)
    self._strings = []



  def euclideanSim(self, x, y, dimensions: int):
    '''
    This function calculates how close two points are using euclidean distance.

    Euclidean distance isn't anything fancy; it's the most basic method for comparing the distance between two points.
    You may have seen it being used like this: √((x2 - x1)²  +  (y2 - y1)²) for measuring distances in 2-D.

    Your function should do the same thing, but in nDim dimensions instead.

    Keep in mind that this function is meant to return similarity i.e the opposite of distance. (This isn't mandatory; just rename the function to avoid confusion if you'd rather just have it return distances)
    After all, if two points are close to each other, their respective texts must be similar in meaning.
    '''
    #Your code goes here!
    diff = x[:dimensions] - y[:dimensions]
    distance = numpy.sqrt(numpy.sum(diff ** 2))

# Convert distance to similarity (smaller distance = higher similarity)
    similarity = 1 / (1 + distance)

    return similarity



  def search(self, input : str, n_return = 1):
    '''
    This is the biggest function here by far.
    This functions job is to find the n closest to the input in our database

    This will be done by generating an embedding for our input, and finding the n closest points to it and what pieces of text are associated to them.

    It should return a tuple (tRText, tRSim).
    tRText will contain the n closest pieces of text and tRSim will contain their respective similarities to the query
    '''
    tRText = []
    tRSim = numpy.array([])

    #Your code goes here!
    query_embedding = self.embedModel.encode([input])[0]

    similarities = []

# Compare input embedding with all stored embeddings
    for stored_embedding in self._embeddings:
        sim = self.euclideanSim(query_embedding, stored_embedding, self.nDim)
        similarities.append(sim)

    similarities = numpy.array(similarities)

# Get indices of top N most similar
    top_indices = numpy.argsort(-similarities)[:n_return]

# Fetch top N texts and their similarity scores
    tRText = [self._strings[i] for i in top_indices]
    tRSim = similarities[top_indices]

    return (tRText, tRSim)


    return(tRText, tRSim)


In [16]:
vDB = VectorDB(model_name = EMBEDDING_MODEL)
#this code calls the constructor and sets vDB to be an instance of our vector database.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def chunksFromText(text : str):
  '''
  Your goal for this function is to convert the text of a page to individual chunks and store these chunks in our vector database.

  Q: What do CHUNK_SIZE and CHUNK_OVERLAP mean?

  A: CHUNK_SIZE is just the number of characters we are storing in each chunk. CHUNK_OVERLAP refers to the number of characters
  shared between sequential chunks. I.E for a chunk overlap of 50 the first chunk will store store characters 0 to 500, the second 450 to 950,
  the third 900 to 1400 and so on. Feel free to play around with these values; I chose them at random.

  Also note that chunking by character count isn't the only approach that you can use. Another method is to have each chunk be a sentence,
  multiple sentences, or an entire paragraph. Again, feel free to experiment!
  '''

  CHUNK_SIZE = 500
  CHUNK_OVERLAP = 50

  docChunks = []

  #Your code goes here!
  i = 0
  while i < len(text):
      chunk = text[i:i + CHUNK_SIZE]
      docChunks.append(chunk)
      i += CHUNK_SIZE - CHUNK_OVERLAP

  vDB.addToDatabase(input=docChunks)


In [15]:
def chunksFromPDF(path = "", startPage = 0, endPage = None):
  '''
  Here's the documentation for the PDFReader library: https://pdfreader.readthedocs.io/en/latest/tutorial.html
  Your goal for this function is to extract text from each page in the PDF and call chunksFromText on the text to convert it to chunks and store
  it in the vector database.

  Quick reference: pdf = PDFReader(path) #gets us a a PDFReader object that we can call functions on
                   pdf.pages #big array containing all the pages
                   page = pdf.pages[0] #gets first page
                   text = page.extract_text() #gets text from the page

  You are required to clean the text from each page, as well as decide which pages are actually relevant.
  Examples of things you should get rid of (Not exhaustive):
  1. Headers, footers,
  2. Indexes,
  3. Title page,
  etc.
  '''
  #Your code goes here!
  pdf = PDFReader(path)
  pages = pdf.pages

  if endPage is None:
        endPage = len(pages)

  full_text = ""

  for i in range(startPage, endPage):
        page = pages[i]
        text = page.extract_text()

        if text:
            # Optional cleanup
            text = text.strip()
            full_text += text + " "

  chunksFromText(full_text)




In [17]:
chunksFromPDF("Understanding Stocks.pdf", startPage=5)

In [ ]:
answerBot = pipeline("text-generation", model = LLM_MODEL, trust_remote_code=True) # This bit of code loads our LLM into memory; we can access it using
                                                                                   # the answerBot object.
                                                                                   #The model we are using is 2.2GB, so make sure you've got a decent WiFi connection.

In [26]:
def generateAnswer(question: str, n_chunks = 3):
    # Get top relevant chunks
    context, _ = vDB.search(input=question, n_return=n_chunks)

    # Build the full prompt
    context_text = "\n".join(context)
    prompt = f"""Use the following context to answer the question:

Context:
{context_text}

Question:
{question}

Answer:"""

    # Load model pipeline
    rag_pipeline = pipeline("text-generation", model=LLM_MODEL)

    # Generate answer
    result = rag_pipeline(prompt, max_new_tokens=200)[0]["generated_text"]

    return result


In [28]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --ClearOutputPreprocessor.enabled=True --to notebook --output project3_clean.ipynb project3_chatbot_final.ipynb

[NbConvertApp] WARNING | pattern 'project3_chatbot_final.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--

In [27]:
print(generateAnswer("What is a 401k?"))

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


Use the following context to answer the question:

Context:
untary tax-deferred savings plan that is provided by a number of companies. The popular 401(k) plan is one of the reasons so 
many people became involved in the stock market to begin with. The brilliant part of the 401(k) is that you don’t have to pay taxes on the money you earn until you are 591⁄2. If you leave the company 
before you’re 591⁄2, you can convert your 401(k) to an IRA, another type of tax-deferred savings plan. 
Why People Choose Mutual Funds 
The main reason that people choose mu
le from trading mutual funds in taxable accounts, many have allowed 401(k) and 403(b) plan participants to continue to use this strategy. To my knowledge, no one has ever revealed this 
secret. 
file:///D|/Ebooks/Business/Stock/Understandingstocks/Understanding%20Stocks.htm (83 of 91)10/29/2006 2:46:25 PM file:///D|/Ebooks/Business/Stock/Understandingstocks/Understanding%20Stocks.htm
The Rules 
1.  You begin with a company-sponsored 40

📌 Wrapping Up

And that should be it! Try calling the generate answer function we just defined with a finance related question.

I'd recommend asking it what a 401(k) is. The PDF we used uses a fancy bit of unicode to write 59.5 (which is how old you have to be before you start paying taxes on a 401k), and if our code is working properly, the LLM will work that character into its answer.

I hope you had fun working on this project and learned something new